<a href="https://colab.research.google.com/github/LahiruRajapaksha/cardiac-arrest-prediction-using-bed-head-tickets/blob/colab-code/Cardiac_Arrest_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler


# Breaking the data set as (112,52,20)'
newPatient_list = np.array([])

def dataGroupingAndPadding(patient_data,time_step,no_of_features,no_of_patients):
    scalar = MinMaxScaler(feature_range=(0, 1))  # Scalar object is using in two places
    zeros = np.zeros(no_of_features)
    patient_groups = patient_data.groupby('ID')

    newPatient_list = []
    decision_tree_data = []
    for x in range(no_of_patients):
        patient_records = patient_groups.get_group((x + 1))
        patient_records = np.array(patient_records)
        labels = patient_records[:, -1]
        decision_tree_data = np.append(decision_tree_data, patient_records[0])
        no_of_records = patient_records.shape[0]

        if (time_step - no_of_records) > 0:
            # Create a zero-filled array with the same number of features as the existing records (excluding the last column)
            zeros = np.zeros((time_step - no_of_records, patient_records.shape[1]-1))
            # Create an array with the last column (label) repeated for the zero-filled records
            labels_repeated = np.repeat(labels[-1], time_step - no_of_records)
            labels_repeated = np.expand_dims(labels_repeated, axis=1)
            # Create an array with the last column (label) repeated for the zero-filled records
            padded_records = np.concatenate((np.hstack((zeros, labels_repeated)), patient_records))
        else:
            padded_records = patient_records

        # Select the last 'time_step' records
        padded_records = padded_records[-time_step:]

        padded_records = scalar.fit_transform(padded_records.astype('float32'))

        # Append the padded records for this patient to the list
        newPatient_list.append(padded_records)

    decision_tree_data = decision_tree_data.reshape(no_of_patients, no_of_features)
    newPatient_list = np.array(newPatient_list)
    return newPatient_list, decision_tree_data

In [3]:
%pip install sdv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 118.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4


In [5]:
import pandas as pd
from sdv.metadata import SingleTableMetadata
from sdv.sequential import PARSynthesizer

# Importing the patient records
patient_record = pd.read_csv('/content/sample_data/InitialTrainData/TrainDead.csv')
# patient_record = pd.read_csv('/content/sample_data/InitialTrainData/TrainSurvived.csv')

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=patient_record)
metadata.update_column(
    column_name='ID',
    sdtype='id')
metadata.set_sequence_key(column_name='ID')

synthesizer = PARSynthesizer(
    metadata,
    context_columns=['Age','Gender','Alcoholic','Smoke','FHCD','TriageScore','Outcome'],
    epochs=90,
    verbose=True,
    )

synthesizer.fit(patient_record)

# According to the experiments you need to change the number of sequences
# to define how many sequences you want to generate
synthetic_data = synthesizer.sample(num_sequences=74)


ContextualVersionConflict: ignored